## ESNLI DistilBERT multiclass
In this notebook we examine the performance of interpretability techniques in the ESNLI dataset using DistilBERT on token level 

In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, average_precision_score
from dataset import Dataset
from myModel import MyModel, MyDataset
from myExplainers import MyExplainer
from myEvaluation import MyEvaluation
from sklearn.preprocessing import maxabs_scale
import pickle
from tqdm import tqdm
import datetime
import csv
import warnings
import torch
import tensorflow as tf
from scipy.special import softmax
from helper import print_results, print_results_ap

Loading model and dataset, defining transformer model, and if rationales are available in the dataset

In [2]:
data_path = '../datasets/esnli_multiclass.pickle'
model_path = 'Trained Models/'
save_path = 'Results/esnli_multiclass/'

In [3]:
model_name = 'distilbert'
dataset_name='esnli_distilbert_uncased_multiclass'
existing_rationales = True

Load MyModel, and the subsequent tokenizer

In [5]:
task = 'single_label'
sentence_level = False
labels = 3

model = MyModel(model_path, dataset_name, model_name, task, labels, cased=False, attention=True)
model_no_attention = MyModel(model_path, dataset_name, model_name, task, labels, cased=False, attention=False)
max_sequence_len = model.tokenizer.max_len_single_sentence
tokenizer = model.tokenizer

import torch
print(torch.cuda.is_available())
model.trainer.model.to('cuda')

True


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [6]:
esnli = Dataset(path = data_path)
dataset, label_names = esnli.load_esnli_multiclass()

In [7]:
train_texts, test_texts, test_rationales, validation_texts, train_labels, test_labels, validation_labels = dataset

In [8]:
for i in range(len(test_rationales)):
    if (test_rationales[i][0] == []) & ((test_rationales[i][1] == [])):
        test_rationales[i][0] = [0] * len(test_rationales[i][2])
        test_rationales[i][1] = [0] * len(test_rationales[i][2])
        test_rationales[i][2] = list(test_rationales[i][2])
    
    elif (test_rationales[i][0] == []) & (test_rationales[i][2] == []):
        test_rationales[i][0] = [0] * len(test_rationales[i][1])
        test_rationales[i][1] = list(test_rationales[i][1])
        test_rationales[i][2] = [0] * len(test_rationales[i][1])
    else:
        test_rationales[i][0] = list(test_rationales[i][0])
        test_rationales[i][1] = [0] * len(test_rationales[i][0])
        test_rationales[i][2] = [0] * len(test_rationales[i][0])

C:\Users\chris\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
C:\Users\chris\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  import sys


In [9]:
test_test_rationales = test_rationales

In [10]:
new_rationale = []
len_test = len(test_labels) # 2000
num_labels = len(np.unique(test_labels)) #3

for i in range(len_test):
    rationale = []
    test_t = test_texts[i].split(' ')
    for j in range(num_labels):
        label_rational = []
        for k in range(len(test_t)):
            for r in tokenizer.tokenize(test_t[k]):
                #if r == '.':
                #    print(r)
                #print(r)
                rationall = 1 if test_test_rationales[i][j][k] > 0 else 0
                label_rational.append(rationall)
        rationale.append(label_rational)
    new_rationale.append(rationale)

Then, we measure the performance of the model using f1 score (both macro)

In [11]:
predictions = []
for test_text in test_texts:
    outputs = model.my_predict(test_text)
    predictions.append(outputs[0])

pred_labels = []
for prediction in predictions:
    pred_labels.append(np.argmax(softmax(prediction)))

accuracy_score(test_labels, pred_labels), f1_score(test_labels, pred_labels, average='macro'), f1_score(test_labels, pred_labels, average='micro')

2000it [01:22, 21.57it/s]            

(0.7625, 0.7634691388561317, 0.7625)

2000it [01:40, 21.57it/s]

In [33]:
my_explainers = MyExplainer(label_names, model_no_attention) #model 2

my_evaluators = MyEvaluation(label_names, model_no_attention.my_predict, sentence_level = False, task = 'multi-class', evaluation_level_all = True, tokenizer=tokenizer) #model 2
my_evaluatorsP = MyEvaluation(label_names, model_no_attention.my_predict, sentence_level = False, task = 'multi-class', evaluation_level_all = False, tokenizer=tokenizer) #model 2
evaluation =  {'F':my_evaluators.faithfulness, 'FTP': my_evaluators.faithful_truthfulness_penalty, 
          'NZW': my_evaluators.nzw, 'AUPRC': my_evaluators.auprc}
evaluationP = {'F':my_evaluatorsP.faithfulness, 'FTP': my_evaluatorsP.faithful_truthfulness_penalty, 
          'NZW': my_evaluatorsP.nzw, 'AUPRC': my_evaluators.auprc}

In [13]:
import time
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    now = datetime.datetime.now()
    file_name = save_path + 'ESNLI_DISTILBERT_IG_'+str(now.day) + '_' + str(now.month) + '_' + str(now.year)
    metrics = {'F':[], 'FTP':[], 'NZW':[], 'AUPRC' : []}
    metricsP = {'F':[], 'FTP':[], 'NZW':[], 'AUPRC' : []}
    time_r = []
    my_explainers.neighbours = 2000
    techniques = [my_explainers.ig] #my_explainers.lime 
    for ind in tqdm(range(len(test_texts))):
        torch.cuda.empty_cache() 
        test_rational = new_rationale[ind]
        instance = test_texts[ind]
        my_evaluators.clear_states()
        my_evaluatorsP.clear_states()
        prediction, _, _ = model_no_attention.my_predict(instance)
        enc = model.tokenizer([instance,instance], truncation=True, padding=True)[0]
        mask = enc.attention_mask
        tokens = enc.tokens
    
        interpretations = []
        kk = 0
        for technique in techniques:
            ts = time.time()
            temp = technique(instance, prediction, tokens, mask, _, _)
            interpretations.append([np.array(i)/np.max(abs(np.array(i))) for i in temp])
            time_r.append(time.time()-ts)
            kk = kk + 1
        for metric in metrics.keys():
            evaluated = []
            for interpretation in interpretations:
                evaluated.append(evaluation[metric](interpretation, _, instance, prediction, tokens, _, _, test_rational))
            metrics[metric].append(evaluated)
        my_evaluatorsP.saved_state = my_evaluators.saved_state.copy()
        my_evaluators.clear_states()
        for metric in metrics.keys():
            evaluatedP = []
            for interpretation in interpretations:
                evaluatedP.append(evaluationP[metric](interpretation, _, instance, prediction, tokens, _, _, test_rational))
            metricsP[metric].append(evaluatedP)
with open(file_name+'(A).pickle', 'wb') as handle:
    pickle.dump(metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(file_name+'(P).pickle', 'wb') as handle:
    pickle.dump(metricsP, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(file_name+'_TIME.pickle', 'wb') as handle:
    pickle.dump(time_r, handle, protocol=pickle.HIGHEST_PROTOCOL)
time_r = np.array(time_r)
# time_r.mean()
# time_r.mean(axis=1)

100%|██████████| 2000/2000 [30:34<00:00,  1.09it/s]


We present the results for IG

In [14]:
print(time_r)

[0.46600056 0.36051416 0.35400033 ... 0.55900192 0.41000009 0.44750834]


In [15]:
print_results(file_name+'(A)', [' IG '], metrics, label_names) #[' LIME', ' IG  ']

F
 IG   0.05561000108718872 | 0.07344 0.10423 -0.01085
FTP
 IG   0.2183 | 0.21012 0.23908 0.2057
NZW


c:\Users\chris\Anaconda3\envs\optimus_transformers\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
c:\Users\chris\Anaconda3\envs\optimus_transformers\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


 IG   1.0 | 1.0 1.0 1.0
AUPRC
 IG   0.30086 | 0.40545 0.19499 0.30215


In [16]:
print_results(file_name+'(P)', [' IG '], metricsP, label_names) #[' LIME', ' IG  ']

F
 IG   0.25666 | 0.32222 0.32187 0.12588
FTP
 IG   0.38458 | 0.4319 0.52199 0.19984
NZW
 IG   1.0 | 1.0 1.0 1.0
AUPRC
 IG   0.30086 | 0.40545 0.19499 0.30215


Then, we perform the experiments for the different attention setups!

In [17]:
conf = []
for ci in ['Mean', 'Multi'] + list(range(6)):
    for ce in ['Mean'] + list(range(12)):
        for cp in ['From', 'To', 'MeanColumns', 'MaxColumns']: # Matrix: From, To, MeanColumns, MeanRows, MaxColumns, MaxRows
            for cl in [False]: # Selection: True: select layers per head, False: do not
                conf.append([ci, ce, cp, cl])
len(conf)

416

In [18]:
import time 
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    now = datetime.datetime.now()
    
    file_name = save_path + 'ESNLI_DISTILBERT_ATTENTION_'+str(now.day) + '_' + str(now.month) + '_' + str(now.year)
    metrics = {'FTP':[], 'F':[], 'NZW':[], 'AUPRC' : []}
    metricsP = {'FTP':[], 'F':[], 'NZW':[], 'AUPRC' : []}
    time_r = []
    time_b = []
    time_b2 = []
    for con in conf:
        time_r.append([])
    for ind in tqdm(range(len(test_texts))):
        torch.cuda.empty_cache() 
        test_rational = new_rationale[ind]
        instance = test_texts[ind]
        my_evaluators.clear_states()
        my_evaluatorsP.clear_states()
        my_explainers.save_states = {}
        prediction, attention, _ = model.my_predict(instance)
        enc = model.tokenizer([instance,instance], truncation=True, padding=True)[0]
        mask = enc.attention_mask
        tokens = enc.tokens
        
        interpretations = []
        kk = 0
        for con in conf:
            ts = time.time()
            my_explainers.config = con
            temp = my_explainers.my_attention(instance, prediction, tokens, mask, attention, _)
            interpretations.append([maxabs_scale(i) for i in temp])
            time_r[kk].append(time.time()-ts)
            kk = kk + 1
        for metric in metrics.keys():
            evaluated = []
            k = 0
            for interpretation in interpretations:
                tt = time.time()
                evaluated.append(evaluation[metric](interpretation, _, instance, prediction, tokens, _, _, test_rational))
                k = k + (time.time()-tt)
            if metric == 'FTP':
                time_b.append(k)
            metrics[metric].append(evaluated)
        my_evaluatorsP.saved_state = my_evaluators.saved_state.copy()
        for metric in metrics.keys():
            evaluated = []
            k = 0
            for interpretation in interpretations:
                tt = time.time()
                evaluated.append(evaluationP[metric](interpretation, _, instance, prediction, tokens, _, _, test_rational))
                k = k + (time.time()-tt)
            if metric == 'FTP':
                time_b2.append(k)
            metricsP[metric].append(evaluated)
with open(file_name+' (A).pickle', 'wb') as handle:
    pickle.dump(metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(file_name+' (P).pickle', 'wb') as handle:
    pickle.dump(metricsP, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(file_name+'_TIME.pickle', 'wb') as handle:
    pickle.dump(time_r, handle, protocol=pickle.HIGHEST_PROTOCOL)
time_r = np.array(time_r)
# time_r.mean(axis=1).min(),time_r.mean(axis=1).max(), time_r.mean(axis=1).mean(), time_r.sum(axis=1).mean(), np.mean(time_b), np.mean(time_b2)

  0%|          | 0/2000 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [1:41:51<00:00,  3.06s/it]


We present the results of the different attention setups

In [19]:
try:
    time_r.mean(axis=1).min(),time_r.mean(axis=1).max(), time_r.mean(axis=1).mean(), time_r.sum(axis=1).mean(), np.mean(time_b), np.mean(time_b2)
except:
    print('Failure')

In [ ]:
print_results(file_name+' (A)', conf, metrics, label_names)

In [ ]:
print_results(file_name+' (P)', conf, metricsP, label_names)

FTP
['Mean', 'Mean', 'From', False]  0.27349 | 0.46979 0.07385 0.27682
['Mean', 'Mean', 'To', False]  0.20412 | 0.31036 0.1098 0.1922
['Mean', 'Mean', 'MeanColumns', False]  0.21234 | 0.34268 0.12827 0.16607
['Mean', 'Mean', 'MaxColumns', False]  0.23337 | 0.37289 0.1227 0.20452
['Mean', 0, 'From', False]  0.25855 | 0.43783 0.07917 0.25864
['Mean', 0, 'To', False]  0.2606 | 0.4181 0.07936 0.28433
['Mean', 0, 'MeanColumns', False]  0.173 | 0.27536 0.10598 0.13766
['Mean', 0, 'MaxColumns', False]  0.1933 | 0.28671 0.1125 0.1807
['Mean', 1, 'From', False]  0.29034 | 0.49044 0.08323 0.29735
['Mean', 1, 'To', False]  0.1578 | 0.24991 0.08381 0.13969
['Mean', 1, 'MeanColumns', False]  0.27247 | 0.4362 0.14445 0.23676
['Mean', 1, 'MaxColumns', False]  0.2725 | 0.43652 0.12847 0.25251
['Mean', 2, 'From', False]  0.20768 | 0.32499 0.0808 0.21725
['Mean', 2, 'To', False]  0.22867 | 0.31233 0.12931 0.24437
['Mean', 2, 'MeanColumns', False]  0.14125 | 0.1847 0.12348 0.11557
['Mean', 2, 'MaxColumns

We calculate the best attention setup using Optimus variations (we do not use the Optimus implementation at this step)

In [21]:
print_results_ap(metrics, label_names, conf)

Baseline: -9.642239969428108e-10  and NZW: 1.0 and AUPRC: 0.5758322755456599
Max Across: 8.773818754859164e-10  and NZW: 1.0 and AUPRC: 0.2761847210876263
Per Label Per Instance: 0.10987035626892173  and NZW:  0.9987880039231168 and AUPRC: 0.33263783808001857
Per Instance: 5.006286582509567e-08  and NZW:  0.9998266363858749 and AUPRC: 0.26100632675238006


In [22]:
print_results_ap(metricsP, label_names, conf)

Baseline: 0.4807783246179615  and NZW: 1.0 and AUPRC: 0.5758322755456599
Max Across: 0.5521760721678511  and NZW: 0.9658337671927848 and AUPRC: 0.6512284945949852
Per Label Per Instance: 0.16472461552748727  and NZW:  0.9958414055143497 and AUPRC: 0.557975745153999


c:\Users\chris\Anaconda3\envs\optimus_transformers\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


Per Instance: 0.705810381210702  and NZW:  0.9958414055143497 and AUPRC: 0.49840872100485384


We repeat the process with Attention Scores with negative values (A*), thus by skipping the Softmax function. In the attention setups, we exclude the multiplication option in heads and layers, as a few combinations reach +/-inf

In [34]:
conf = []
for ci in ['Mean'] + list(range(6)):
    for ce in ['Mean'] + list(range(12)):
        for cp in ['From', 'To', 'MeanColumns', 'MaxColumns']: # Matrix: From, To, MeanColumns, MeanRows, MaxColumns, MaxRows
            for cl in [False]: # Selection: True: select layers per head, False: do not
                conf.append([ci, ce, cp, cl])
len(conf)

364

In [35]:
import time 
import math
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    now = datetime.datetime.now()
    
    file_name = save_path + 'ESNLI_DISTILBERT_A_ATTENTION_NO_SOFTMAX_'+str(now.day) + '_' + str(now.month) + '_' + str(now.year)
    metrics = {'FTP':[], 'F':[], 'NZW':[], 'AUPRC' : []}
    metricsP = {'FTP':[], 'F':[], 'NZW':[], 'AUPRC' : []}
    time_r = []
    time_b = []
    time_b2 = []
    for con in conf:
        time_r.append([])
    for ind in tqdm(range(len(test_texts))):
        torch.cuda.empty_cache() 
        test_rational = new_rationale[ind]
        instance = test_texts[ind]
        my_evaluators.clear_states()
        my_evaluatorsP.clear_states()
        my_explainers.save_states = {}
        prediction, _, hidden_states = model.my_predict(instance)
        enc = model.tokenizer([instance,instance], truncation=True, padding=True)[0]
        mask = enc.attention_mask
        tokens = enc.tokens
        
        attention = []
        for la in range(6):
            our_new_layer = []
            bob =  model.trainer.model.base_model.transformer.layer[la].attention
            has = hidden_states[la]
            aaa = bob.k_lin(torch.tensor(has).to('cuda'))
            bbb = bob.q_lin(torch.tensor(has).to('cuda'))
            for he in range(12):
                bbb = bbb / math.sqrt(64)
                attention_scores = torch.matmul(bbb[:,he*64:(he+1)*64], aaa[:,he*64:(he+1)*64].transpose(-1, -2))
                our_new_layer.append(attention_scores.cpu().detach().numpy())
            attention.append(our_new_layer)
        attention = np.array(attention)
        
        interpretations = []
        kk = 0
        for con in conf:
            ts = time.time()
            my_explainers.config = con
            temp = my_explainers.my_attention(instance, prediction, tokens, mask, attention, _)
            interpretations.append([maxabs_scale(i) for i in temp])
            time_r[kk].append(time.time()-ts)
            kk = kk + 1
        for metric in metrics.keys():
            evaluated = []
            k = 0
            for interpretation in interpretations:
                tt = time.time()
                evaluated.append(evaluation[metric](interpretation, _, instance, prediction, tokens, _, _, test_rational))
                k = k + (time.time()-tt)
            if metric == 'FTP':
                time_b.append(k)
            metrics[metric].append(evaluated)
        my_evaluatorsP.saved_state = my_evaluators.saved_state.copy()
        for metric in metrics.keys():
            evaluated = []
            k = 0
            for interpretation in interpretations:
                tt = time.time()
                evaluated.append(evaluationP[metric](interpretation, _, instance, prediction, tokens, _, _, test_rational))
                k = k + (time.time()-tt)
            if metric == 'FTP':
                time_b2.append(k)
            metricsP[metric].append(evaluated)        
with open(file_name+' (A).pickle', 'wb') as handle:
    pickle.dump(metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(file_name+' (P).pickle', 'wb') as handle:
    pickle.dump(metricsP, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(file_name+'_TIME.pickle', 'wb') as handle:
    pickle.dump(time_r, handle, protocol=pickle.HIGHEST_PROTOCOL)
time_r = np.array(time_r)
time_r.mean(axis=1).min(),time_r.mean(axis=1).max(), time_r.mean(axis=1).mean(), time_r.sum(axis=1).mean(), np.mean(time_b), np.mean(time_b2)

100%|██████████| 2000/2000 [1:40:48<00:00,  3.02s/it]


We present the results for the different attention setups

In [37]:
print_results(file_name+' (A)', conf, metrics, label_names)

FTP
['Mean', 'Mean', 'From', False]  0.0 | 0.06155 -0.08109 0.01955
['Mean', 'Mean', 'To', False]  0.0 | 0.06824 0.06061 -0.12885


c:\Users\chris\Anaconda3\envs\optimus_transformers\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
c:\Users\chris\Anaconda3\envs\optimus_transformers\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


['Mean', 'Mean', 'MeanColumns', False]  0.0 | 0.0547 -0.20871 0.154
['Mean', 'Mean', 'MaxColumns', False]  -0.0 | -0.02989 0.10444 -0.07456
['Mean', 0, 'From', False]  0.0 | 0.06175 -0.07788 0.01613
['Mean', 0, 'To', False]  -0.0 | 0.06206 0.06405 -0.12611
['Mean', 0, 'MeanColumns', False]  0.0 | 0.05397 -0.20811 0.15414
['Mean', 0, 'MaxColumns', False]  -0.0 | -0.0295 0.10231 -0.07281
['Mean', 1, 'From', False]  -0.0 | 0.07041 -0.11127 0.04086
['Mean', 1, 'To', False]  0.0 | 0.12152 -0.05087 -0.07065
['Mean', 1, 'MeanColumns', False]  0.0 | 0.04256 -0.18042 0.13786
['Mean', 1, 'MaxColumns', False]  -0.0 | -0.02172 0.12072 -0.09901
['Mean', 2, 'From', False]  0.0 | 0.04808 -0.09204 0.04396
['Mean', 2, 'To', False]  -0.0 | 0.03266 0.14499 -0.17765
['Mean', 2, 'MeanColumns', False]  0.0 | 0.00085 -0.18392 0.18307
['Mean', 2, 'MaxColumns', False]  -0.0 | -0.03089 0.08608 -0.0552
['Mean', 3, 'From', False]  -0.0 | 0.0473 -0.07943 0.03213
['Mean', 3, 'To', False]  -0.0 | 0.0594 0.12363 -0.1

In [ ]:
print_results(file_name+' (P)', conf, metricsP, label_names)

FTP
['Mean', 'Mean', 'From', False]  0.06278 | 0.17499 -0.07977 0.09313
['Mean', 'Mean', 'To', False]  0.24428 | 0.37958 0.09993 0.25333
['Mean', 'Mean', 'MeanColumns', False]  -0.01523 | -0.02116 0.0259 -0.05045
['Mean', 'Mean', 'MaxColumns', False]  0.15578 | 0.21461 0.13668 0.11605
['Mean', 0, 'From', False]  0.0811 | 0.1946 -0.05802 0.10672
['Mean', 0, 'To', False]  0.24514 | 0.38113 0.09951 0.2548
['Mean', 0, 'MeanColumns', False]  0.00324 | 0.00633 0.03482 -0.03143
['Mean', 0, 'MaxColumns', False]  0.15331 | 0.21218 0.13354 0.11423
['Mean', 1, 'From', False]  -0.03556 | 0.05055 -0.13913 -0.01811
['Mean', 1, 'To', False]  0.19701 | 0.32684 0.0944 0.16978
['Mean', 1, 'MeanColumns', False]  -0.09883 | -0.14493 -0.05706 -0.09449
['Mean', 1, 'MaxColumns', False]  0.21614 | 0.3506 0.14821 0.14963
['Mean', 2, 'From', False]  -0.08031 | -0.0647 -0.13954 -0.03671
['Mean', 2, 'To', False]  0.24273 | 0.38284 0.10358 0.24177
['Mean', 2, 'MeanColumns', False]  -0.20795 | -0.31423 -0.10623 -0.

We calculate the best attention setup using Optimus variations (we do not use the Optimus implementation script at this step)

In [38]:
print_results_ap(metrics, label_names, conf)

Baseline: 2.9209847272430406e-10  and NZW: 1.0 and AUPRC: 0.5689713873592724
Max Across: 1.8596061449975092e-09  and NZW: 1.0 and AUPRC: 0.33278931316539645
Per Label Per Instance: 0.2293016406751467  and NZW:  0.8688194382926696 and AUPRC: 0.32143193834823014
Per Instance: 1.1497241877974605e-07  and NZW:  0.885176830650726 and AUPRC: 0.2696148778389646


In [39]:
print_results_ap(metricsP, label_names, conf)

Baseline: -0.030220227367617095  and NZW: 1.0 and AUPRC: 0.5689713873592724
Max Across: 0.5323738346512958  and NZW: 0.9977543306740405 and AUPRC: 0.5908404817172931
Per Label Per Instance: 0.16286174186313299  and NZW:  0.9540870051259328 and AUPRC: 0.5532768624187631


c:\Users\chris\Anaconda3\envs\optimus_transformers\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


Per Instance: 0.6975678112765703  and NZW:  0.9540870051259328 and AUPRC: 0.48819467246493303
